In [ ]:
from importlib import import_module
import math as m
import os
import sys

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import numpy as np
import seaborn as sb

In [ ]:
# Specify the run ID (aka problem name).
runid = "linecurrent_BxByBz"

# Add the subdirectory for the run results to the module search path.
run_path = os.path.join(".", runid)
sys.path.append(run_path)

# Import the problem definition from the run results directory.
p = import_module(runid)

# Read the run hyperparameters from the run results directory.
import hyperparameters as hp

In [ ]:
# Compute the number of models.
n_models = p.n_var

# Compute the number of rows and columns for the 2-per-row plot.
n_cols = 2
n_rows = m.ceil(n_models/n_cols)

# Compute the figure size for model loss plots, assuming 5x5 for each figure, in rows of 2 plots.
loss_figsize = (5*n_cols, 5*n_rows)

# Compute the figure size for heat maps, assuming 6x4 for each figure, in rows of 2 plots.
# heatmap_figsize = (12*n_rows, 4*n_rows)

# Compute the figure size for the start-and-end comparison plots, in rows of 2 plots.
# start_end_figsize = (12*n_rows, 4*n_rows)

In [ ]:
# Analytical solution

# Constants
mu0 = 1.0  # Normalized vacuum permittivity.
I = 1.0    # Normalized current.
Q = 60.0   # Flow angle in degrees clockwise from +y axis.
u0 = 1.0   # Initial flow speed.

# Compute the constant velocity components.
ux = u0*np.sin(np.radians(Q))
uy = u0*np.cos(np.radians(Q))

def Bx_analytical(t, x, y):
    r = np.sqrt((x - ux*t)**2 + (y - uy*t)**2)
    Bx = -mu0*I/(2*np.pi)*(y - uy*t)/r**2
    return Bx

def By_analytical(t, x, y):
    r = np.sqrt((x - ux*t)**2 + (y - uy*t)**2)
    By = mu0*I/(2*np.pi)*(x - ux*t)/r**2
    return By

def Bz_analytical(t, x, y):
    Bz = np.zeros(t.shape)
    return Bz

In [ ]:
# Load the training points.
txy_train = np.loadtxt(os.path.join(runid, "X_train.dat"))
t_train = txy_train[:, 0]
x_train = txy_train[:, 1]
y_train = txy_train[:, 2]

# Compute the data limits.
t_min = t_train[0]
t_max = t_train[-1]
x_min = x_train[0]
x_max = x_train[-1]
y_min = y_train[0]
y_max = y_train[-1]

In [ ]:
# Extract the unique training point values.
t_train_vals = np.unique(t_train)
x_train_vals = np.unique(x_train)
y_train_vals = np.unique(y_train)
n_t_train_vals = len(t_train_vals)
n_x_train_vals = len(x_train_vals)
n_y_train_vals = len(y_train_vals)

In [ ]:
# Compute the heat map tick locations and labels.
n_x_ticks = 6
x_tick_pos = np.linspace(0, n_x_train_vals - 1, n_x_ticks)
x_tick_labels = ["%.2f" % (x_min + x/(n_x_train_vals - 1)*(x_max - x_min)) for x in x_tick_pos]
n_y_ticks = 6
y_tick_pos = np.linspace(0, n_y_train_vals - 1, n_y_ticks)
y_tick_labels = ["%.2f" % (y_min + y/(n_y_train_vals - 1)*(y_max - y_min)) for y in y_tick_pos]
y_tick_labels = list(reversed(y_tick_labels))

In [ ]:
# Load the data locations and values.
txy_data = np.loadtxt(os.path.join(runid, "XY_data.dat"))

In [ ]:
# Load the model-predicted values.
ψ = []
delψ = []
for i in range(len(p.dependent_variable_names)):
    var_name = p.dependent_variable_names[i]
    ψ.append(np.loadtxt(os.path.join(runid, "%s_train.dat" % var_name)))
    delψ.append(np.loadtxt(os.path.join(runid, "del_%s_train.dat" % var_name)))

In [ ]:
# Load the loss function histories.
losses_model = np.loadtxt(os.path.join(runid, "losses_model.dat"))
losses_model_res = np.loadtxt(os.path.join(runid, "losses_model_res.dat"))
losses_model_data = np.loadtxt(os.path.join(runid, "losses_model_data.dat"))
losses = np.loadtxt(os.path.join(runid, "losses.dat"))
losses_res = np.loadtxt(os.path.join(runid, "losses_res.dat"))
losses_data = np.loadtxt(os.path.join(runid, "losses_data.dat"))

In [ ]:
# Plot the loss history for each model.
plt.figure(figsize=loss_figsize)
for i in range(p.n_var):
    plt.subplot(n_rows, n_cols, i + 1)
    # variable_name = p.dependent_variable_names[i]
    plt.semilogy(losses_model_res[:, i], label="$L_{res}$")
    plt.semilogy(losses_model_data[:, i], label="$L_{data}$")
    plt.semilogy(losses_model[:, i], label="$L$")
    plt.title(p.dependent_variable_labels[i])
    plt.legend()
plt.suptitle("Loss function histories by model")
plt.show()

In [ ]:
# Plot the total loss function history.
plt.semilogy(losses_res, label="$L_{res}$")
plt.semilogy(losses_data, label="$L_{data}$")
plt.semilogy(losses, label="$L$")
plt.xlabel("Epoch")
plt.ylabel("Loss function")
plt.legend()
plt.grid()
plt.title("Loss function evolution for %s" % runid)
plt.show()

In [ ]:
# Plot the actual initial magnetic field as a quiver plot.
x = txy_data[:, 1]
y = txy_data[:, 2]
B0x_act = txy_data[:, 3]
B0y_act = txy_data[:, 4]
plt.quiver(x, y, B0x_act, B0y_act, scale=2)
ax = plt.gca()
ax.set_aspect(1.0)
ax.grid()
plt.title("Initial magnetic field (actual)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
B0x_act.shapedd

In [ ]:
# Plot the actual initial magnetic field as a logarithmic heat map.
# These data were generated with the analytical solution.
B0x_act = txy_data[:, 3]
B0y_act = txy_data[:, 4]
B0_act = np.sqrt(B0x_act**2 + B0y_act**2)
# To get the proper orientation, reshape, transpose, flip.
B0_act = np.flip(B0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0_act, norm=LogNorm(), square=True)
plt.xticks(x_tick_pos, x_tick_labels)
plt.yticks(y_tick_pos, y_tick_labels)
plt.title("Initial magnetic field (actual)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the predicted initial magnetic field as a quiver plot.
n_start = n_x_train_vals*n_y_train_vals
x = x_train[:n_start]
y = y_train[:n_start]
B0x_pred = ψ[0][:n_start]
B0y_pred = ψ[1][:n_start]
plt.quiver(x, y, B0x_pred, B0y_pred)
ax = plt.gca()
ax.set_aspect(1.0)
ax.grid()
plt.title("Initial magnetic field (predicted)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the predicted initial magnetic field as a logarithmic heat map.
n_start = n_x_train_vals*n_y_train_vals
x = x_train[:n_start]
y = y_train[:n_start]
B0x_pred = ψ[0][:n_start]
B0y_pred = ψ[1][:n_start]
B0_pred = np.sqrt(B0x_pred**2 + B0y_pred**2)
# To get the proper orientation, reshape, transpose, flip.
B0_pred = np.flip(B0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0_pred, norm=LogNorm(), square=True)
plt.xticks(x_tick_pos, x_tick_labels)
plt.yticks(y_tick_pos, y_tick_labels)
plt.title("Initial magnetic field (predicted)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the error in the predicted initial magnetic field as a heat map.
n_start = n_x_train_vals*n_y_train_vals
x = x_train[:n_start]
y = y_train[:n_start]
B0_err = (B0_pred - B0_act)/B0_act
# To get the proper orientation, reshape, transpose, flip.
B0_err = np.flip(B0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0_err, square=True)
plt.xticks(x_tick_pos, x_tick_labels)
plt.yticks(y_tick_pos, y_tick_labels)
plt.title("Initial magnetic field (relative error)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the absolute error in the predicted initial Bz as a heat map.
n_start = n_x_train_vals*n_y_train_vals
t = t_train[:n_start]
x = x_train[:n_start]
y = y_train[:n_start]
B0z_pred = ψ[2][:n_start]
B0z_act = Bz_analytical(t, x, y)
B0z_err = B0z_pred - B0z_act
# To get the proper orientation, reshape, transpose, flip.
B0z_err = np.flip(B0z_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B0z_err, square=True)
plt.xticks(x_tick_pos, x_tick_labels)
plt.yticks(y_tick_pos, y_tick_labels)
plt.title("Initial %s (absolute error)" % p.dependent_variable_labels[2])
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the analytical final magnetic field as a quiver plot.
n_end = n_x_train_vals*n_y_train_vals
t = t_train[-n_end:]
x = x_train[-n_end:]
y = y_train[-n_end:]
B1x_act = Bx_analytical(t, x, y)
B1y_act = By_analytical(t, x, y)
plt.quiver(x, y, B1x_act, B1y_act)
ax = plt.gca()
ax.set_aspect(1.0)
ax.grid()
plt.title("Final magnetic field (analytical)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the analytical final magnetic field as a logarithmic heat map.
n_end = n_x_train_vals*n_y_train_vals
t = t_train[-n_end:]
x = x_train[-n_end:]
y = y_train[-n_end:]
B1x_act = Bx_analytical(t, x, y)
B1y_act = By_analytical(t, x, y)
B1_act = np.sqrt(B1x_act**2 + B1y_act**2)
# To get the proper orientation, reshape, transpose, flip.
B1_act = np.flip(B1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1_act, norm=LogNorm(), square=True)
plt.xticks(x_tick_pos, x_tick_labels)
plt.yticks(y_tick_pos, y_tick_labels)
plt.title("Final magnetic field (analytical)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the predicted final magnetic field as a quiver plot.
n_end = n_x_train_vals*n_y_train_vals
x = x_train[-n_end:]
y = y_train[-n_end:]
B1x_pred = ψ[0][-n_end:]
B1y_pred = ψ[1][-n_end:]
plt.quiver(x, y, B1x_pred, B1y_pred)
ax = plt.gca()
ax.set_aspect(1.0)
ax.grid()
plt.title("Final magnetic field (predicted)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the predicted final magnetic field as a logarithmic heat map.
n_end = n_x_train_vals*n_y_train_vals
x = x_train[-n_end:]
y = y_train[-n_end:]
B1x_pred = ψ[0][-n_end:]
B1y_pred = ψ[1][-n_end:]
B1_pred = np.sqrt(B1x_pred**2 + B1y_pred**2)
# To get the proper orientation, reshape, transpose, flip.
B1_pred = np.flip(B1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1_pred, norm=LogNorm(), square=True)
plt.xticks(x_tick_pos, x_tick_labels)
plt.yticks(y_tick_pos, y_tick_labels)
plt.title("Final magnetic field (predicted)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the error in the predicted final magnetic field as a heat map.
n_end = n_x_train_vals*n_y_train_vals
x = x_train[-n_end:]
y = y_train[-n_end:]
B1_err = (B1_pred - B1_act)/B1_act
# To get the proper orientation, reshape, transpose, flip.
B1_err = np.flip(B1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1_err, square=True)
plt.xticks(x_tick_pos, x_tick_labels)
plt.yticks(y_tick_pos, y_tick_labels)
plt.title("Final magnetic field (relative error)")
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()

In [ ]:
# Plot the error in the predicted final Bz as a heat map.
n_end = n_x_train_vals*n_y_train_vals
t = t_train[-n_end:]
x = x_train[-n_end:]
y = y_train[-n_end:]
B1z_pred = ψ[2][-n_end:]
B1z_act = Bz_analytical(t, x, y)
B1z_err = B1z_pred - B1z_act
# To get the proper orientation, reshape, transpose, flip.
B1z_err = np.flip(B1z_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
sb.heatmap(B1z_err, square=True)
plt.xticks(x_tick_pos, x_tick_labels)
plt.yticks(y_tick_pos, y_tick_labels)
plt.title("Final %s (absolute error)" % p.dependent_variable_labels[2])
plt.xlabel(p.independent_variable_labels[1])
plt.ylabel(p.independent_variable_labels[2])
plt.show()